In [1]:
from bs4 import BeautifulSoup
import requests
import pytz
import bson
import pytz
from datetime import datetime
import json

In [2]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.companyDb
collection = db.dumb

In [3]:
#OK
def get_descriptions(articles, descriptions):
    for i in range(0, len(articles)):
        descriptions.append(articles[i].find("div", attrs={"class": "m-object__description"}).text)

        #OK
def get_images(articles, imgs):
    for i in range(0, len(articles)):
        imgs.append(articles[i].find("div", attrs={"class": "m-object__img"}).figure.a.img.attrs.get("data-src"))
#OK
def get_headers(articles, headers):
    for i in  range(0, len(articles)):
        headers.append(articles[i].find("div", attrs={"class": "m-object__body"}).h3.text)

#get contents
def get_contents(articles_links, contents):
    
    for i in range(0, len(articles_links)):
        content = ""
        r = requests.get(articles_links[i])
        source = BeautifulSoup(r.content, "lxml")
        content_type = check_content(source)
        
        if(content_type == "TYPE1"):
            all_ps = source.find("div", attrs={"class":"article__html small-12 large-8 column"}).find_all("p")
            for j in range(0, len(all_ps)):
                text = all_ps[j].text
                content = content + text
            
        elif(content_type == "TYPE2"):
            all_ps = source.find("div", attrs={"class":"c-article-content"}).find_all("p")
            for j in range(0, len(all_ps)):
                text = all_ps[j].text
                content = content + text
            
        elif(content_type == "TYPE3"):
            all_ps = source.find("div", attrs={"class":"c-article-content js-article-content article__content"}).find_all("p")
            for j in range(0, len(all_ps)):
                text = all_ps[j].text
                content = content + text
            
        else:
            print("ERROR")
            
        contents.append(content)
def check_content(source):
    content_type = "TYPE"
    
    #type 1 - "living"
    if(str(source.find("div", attrs={"class":"article__html small-12 large-8 column"})) != "None"):
        content_type = "TYPE1"

    #type 2 - "default"
    elif(str(source.find("div", attrs={"class":"c-article-content js-article-content article__content"})) != "None"):
        content_type = "TYPE3"
    
    #type 3 - "blue colored background"
    elif(str(source.find("div", attrs={"class":"c-article-content"})) != "None"):
        content_type = "TYPE2"
    
    #invalid type
    else:
        content_type = "!TYPE"
        
    return content_type
def get_articles_links(articles, articles_links):
    for i in  range(0, len(articles)):
        link = str("http://www.euronews.com" + str(articles[i].find("div", attrs={"class": "m-object__body"}).h3.a.attrs.get("href")))
        articles_links.append(link)

In [14]:
pageCount = 3
category = "classical-music"

utc = pytz.utc
ist = pytz.timezone('Asia/Calcutta')

descriptions = []
imgs = []
headers = []
contents = []
articles_links = []


for i in range(1, pageCount+1):
    r = requests.get("https://www.euronews.com/tag/"+category+"?p="+str(i))
    source = BeautifulSoup(r.content, "lxml")
    articles = source.find("div", attrs={"class": "o-block-listing__articles"}).find_all("article")
 
    get_descriptions(articles, descriptions)
    get_images(articles, imgs)
    get_headers(articles, headers)
 
    get_articles_links(articles, articles_links)
get_contents(articles_links, contents)

In [15]:
len(descriptions)

78

In [16]:
pop_dict = {}
objeler = []

my_timezone = pytz.timezone("America/Guayaquil")
for i in range(len(descriptions)):
    obje = {}
    obje['title'] = headers[i]
    obje['description'] = descriptions[i]
    obje['content'] = contents[i]
    obje['imageUrl'] = imgs[i]
    obje['rating'] = 0
    obje['isApproved'] = True
    obje['viewCount'] = 0
    obje['category'] = "5df7874a3b36bc3748631a8b"
    aware_datetime = my_timezone.localize(datetime(2002, 10, 27, 6, 0, 0))
    obje['createdAt'] = aware_datetime
    collection.insert_one(obje).inserted_id
